# Collect NLDAS Forcing Data

This notebook demonstrates the process of collecting NLDAS forcing data, regridding these data onto an existing WRF-Hydro (domain collected in the previous notebook), and finally saving the results to HydroShare. 

See this [link](https://hydro1.gesdisc.eosdis.nasa.gov/data/NLDAS/README.NLDAS2.pdf) for details about primary (FORA) and secondary (FORB) NLDAS forcing datasets. The following steps are completed in the https://urs.earthdata.nasa.gov/ website:

1. Create Earthdata account: [link](https://urs.earthdata.nasa.gov/users/new)
2. Link GES DISC with your earthdata account: [link](https://disc.gsfc.nasa.gov/earthdata-login)
3. Select NLDAS2 data via: [https://disc.gsfc.nasa.gov/datasets/NLDAS_FORA0125_H_V002/summary?keywords=NLDAS](https://disc.gsfc.nasa.gov/datasets/NLDAS_FORA0125_H_V002/summary?keywords=NLDAS) and subset the data to a reasonable size and date range using the `Simple Subset Wizard`.  
5. Choose the following variables to extract:
    - 10-m above ground Meridional wind speed
    - 10-m above ground Zonal wind speed 
    - 2-m above ground Specific humidity
    - 2-m above ground Temperature 
    - LW radiation flux downwards
    - Precipitation hourly total
    - Surface pressure
    - SW radiation flux downwards
6. Select "Subset Selected Data Sets" 
7. Select "View Subset Results"
8. Above the list of files, select the "list of URLs" link and copy the contents below

We'll use `bash` to download these data as described [here](https://disc.gsfc.nasa.gov/information/howto/5761bc6a5ad5a18811681bae/how-to-download-data-files-from-http-service-with-wget?keywords=wget). Replace the `<list of URLs>` placholder below with the URL's from Step 7 (above) to write all of the NLDAS URLs to a text file which will be used later to perform a bulk download.

In [ ]:
%%bash 

# make a directory for the raw forcing data
mkdir input_files

# create a file of urls to download
cat > input_files/urls.txt
<list of URLS>

# E.G.
# cat > input_files/urls.txt
#http://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FNLDAS%2FNLDAS_FORA0125_H.002%2F2018%2F213%2FNLDAS_FORA0125_H.A20180801.0000.002.grb&FORMAT=R3JpYi8&BBOX=35.42%2C-96.64%2C44.87%2C-82.49&LABEL=NLDAS_FORA0125_H.A20180801.0000.002.2019078181032.pss.grb&SHORTNAME=NLDAS_FORA0125_H&SERVICE=SUBSET_GRIB&VERSION=1.02&LAYERS=PwM&DATASET_VERSION=002
#http://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FNLDAS%2FNLDAS_FORA0125_H.002%2F2018%2F213%2FNLDAS_FORA0125_H.A20180801.0100.002.grb&FORMAT=R3JpYi8&BBOX=35.42%2C-96.64%2C44.87%2C-82.49&LABEL=NLDAS_FORA0125_H.A20180801.0100.002.2019078181032.pss.grb&SHORTNAME=NLDAS_FORA0125_H&SERVICE=SUBSET_GRIB&VERSION=1.02&LAYERS=PwM&DATASET_VERSION=002
#http://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FNLDAS%2FNLDAS_FORA0125_H.002%2F2018%2F213%2FNLDAS_FORA0125_H.A20180801.0200.002.grb&FORMAT=R3JpYi8&BBOX=35.42%2C-96.64%2C44.87%2C-82.49&LABEL=NLDAS_FORA0125_H.A20180801.0200.002.2019078181032.pss.grb&SHORTNAME=NLDAS_FORA0125_H&SERVICE=SUBSET_GRIB&VERSION=1.02&LAYERS=PwM&DATASET_VERSION=002
#http://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FNLDAS%2FNLDAS_FORA0125_H.002%2F2018%2F213%2FNLDAS_FORA0125_H.A20180801.0300.002.grb&FORMAT=R3JpYi8&BBOX=35.42%2C-96.64%2C44.87%2C-82.49&LABEL=NLDAS_FORA0125_H.A20180801.0300.002.2019078181032.pss.grb&SHORTNAME=NLDAS_FORA0125_H&SERVICE=SUBSET_GRIB&VERSION=1.02&LAYERS=PwM&DATASET_VERSION=002
#http://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FNLDAS%2FNLDAS_FORA0125_H.002%2F2018%2F213%2FNLDAS_FORA0125_H.A20180801.0400.002.grb&FORMAT=R3JpYi8&BBOX=35.42%2C-96.64%2C44.87%2C-82.49&LABEL=NLDAS_FORA0125_H.A20180801.0400.002.2019078181032.pss.grb&SHORTNAME=NLDAS_FORA0125_H&SERVICE=SUBSET_GRIB&VERSION=1.02&LAYERS=PwM&DATASET_VERSION=002
#http://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FNLDAS%2FNLDAS_FORA0125_H.002%2F2018%2F213%2FNLDAS_FORA0125_H.A20180801.0500.002.grb&FORMAT=R3JpYi8&BBOX=35.42%2C-96.64%2C44.87%2C-82.49&LABEL=NLDAS_FORA0125_H.A20180801.0500.002.2019078181032.pss.grb&SHORTNAME=NLDAS_FORA0125_H&SERVICE=SUBSET_GRIB&VERSION=1.02&LAYERS=PwM&DATASET_VERSION=002
#http://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FNLDAS%2FNLDAS_FORA0125_H.002%2F2018%2F213%2FNLDAS_FORA0125_H.A20180801.0600.002.grb&FORMAT=R3JpYi8&BBOX=35.42%2C-96.64%2C44.87%2C-82.49&LABEL=NLDAS_FORA0125_H.A20180801.0600.002.2019078181032.pss.grb&SHORTNAME=NLDAS_FORA0125_H&SERVICE=SUBSET_GRIB&VERSION=1.02&LAYERS=PwM&DATASET_VERSION=002

Follow the instructions [here](https://disc.gsfc.nasa.gov/data-access#mac_linux_wget) for setting up your EarthData account for downloading data. Replace the `<uid>` and `<password>` placeholders below with your EarthData credentials.
 

In [ ]:
%%bash
if [ ! -f ~/.netrc ]; then
    touch ~/.netrc
    echo "machine urs.earthdata.nasa.gov login <uid> password <password>" >> ~/.netrc
    chmod 0600 .netrc
    touch ~/.urs_cookies
fi


Download the files listed in `input_files/urls.txt` using `wget`. This may take several minutes depending on how many files you're downloading. This following command can be executed in a terminal window to count the files as they're downloaded:

```
$watch "ls -l <path to data>/input_files | wc -l"
```

In [ ]:
%%bash

# download these data via wget
cd input_files
wget --load-cookies ~/.urs_cookies \
     --save-cookies ~/.urs_cookies \
     --auth-no-challenge=on \
     --keep-session-cookies \
     --content-disposition \
     -i urls.txt

Preview the data that we downloaded. 

In [ ]:
%%bash

du -h input_files/*

The data that we've downloaded are NLDAS NetCDF files that have been subsetted through time, space, and variable. These data still need to be regridded to the WRF-Hydro domain that was collected in the previous notebook.

Create regridding 'weight' files required by the ESMF regridders. The weight files are netCDF files which specify interpolation weights between the source coordinate data grids (src) and destination coordinate data (dst) grids. The weight file is generated by running the `NLDAS2WRFHydro_generate_weights.ncl` script. We'll need to provide the source and destination grid filenames as arguments to the script, for example:

```
$ ncl interp_opt="bilinear"
    srcGridName=<NLDAS NetCDF File>
    dstGridName=<WRF-Hydro geo_em.d01.nc>
    NLDAS2WRFHydro_generate_weights.ncl
```

This will create the following files:

```
DAS2WRFHydro_weight_bilinear.nc  
PET0.RegridWeightGen.Log  
SCRIP_NLDAS_bilinear.nc  
SCRIP_WRFHydro_bilinear.nc  
```

First, make sure that the NCAR Command Language is installed. Detailed instructions can be found [here](https://www.ncl.ucar.edu/Download/conda.shtml). This following cell will install `ncl` into your conda environment if it doesn't already exist.

In [ ]:
%%bash

# install the ncl language in the conda environment, if it's not already installed.
if  which ncl >/dev/null; then
    echo NCL is already installed!
else
    conda install -y ncl
fi

#!conda install -y ncl

Create the regridding weight file.

In [ ]:
%%bash

FILE=`find input_files -name '*.grb' | head -1`
echo $FILE

ncl 'interp_opt="bilinear"' \
'srcGridName="'$FILE'"' \
'dstGridName="DOMAIN/geo_em.d01.nc"' \
NLDAS2WRFHydro_generate_weights.ncl


Regrid all of the NLDAS files in the `input_files` directory using the NLDAS2WRFHydro_regrid.ncl script. This script takes NLDAS data and a weight file as inputs and outputs regridded data.

In [ ]:
%%bash

ncl 'srcFileName="NLDAS_FORA0125_H.*"'\
 'dstGridName="DOMAIN/geo_em.d01.nc"' \
NLDAS2WRFHydro_regrid.ncl


Clean the our directory by (1) removing the raw NLDAS data that we downloaded from EarthData and (2) rename the default `output_files` directory to `FORCING`.

In [ ]:
!rm -rf input_files
!mv output_files FORCING
!ls FORCING

The next notebook demonstrates how these `FORCING` data can be used to run the WRF-Hydro model. However, execution of the model must be completed on your local machine, so first we'll save our data to HydroShare. Replace the `<abstract>`, `<title>`, and `<keyword>`, placeholders below:

In [ ]:
from utilities import hydroshare
hs = hydroshare.hydroshare()

# compress the DOMAIN and FORCING directories
!tar -czf wrfhydro-data.tar.gz DOMAIN FORCING

# lets save this content as a new resource in HydroShare
abstract = '<abstract>'
title = '<title>'    
keywords = ['<keyword>', '<keyword>2']         

# create the new resource
resource_id = hs.createHydroShareResource(abstract, 
                                          title, 
                                          keywords=keywords, 
                                          resource_type='compositeresource', 
                                          content_files=['wrfhydro-data.tar.gz',
                                                         'WRFHydro-Domain.ipynb',
                                                         'WRFHydro-Forcing.ipynb,
                                                         'WRFHydro-Simulation.ipynb'], 
                                          public=False)